In [ ]:
Param_SemanticModelId= ""

In [ ]:
import sempy.fabric as fabric
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

# Get List of all workspaces
df_workspaces = fabric.list_workspaces().query('Type != "AdminInsights"')

# Get List of all items within the workspaces
df_items = pd.concat(
    [fabric.list_items(workspace=ws_id) for ws_id in df_workspaces.Id],
    ignore_index=True
)

# Get List of all Semantic Models
df_SemanticModels = df_items[df_items["Type"]== "SemanticModel"] # Only these Item Types leverage Connections
df_SemanticModels = df_SemanticModels.merge(df_workspaces[["Id", "Name"]], left_on="Workspace Id", right_on="Id", suffixes=('', '_Workspace'))


# Iterate over workspaces and fetch reports
all_reports = []

for _, row in df_workspaces.iterrows():
    workspace_id = row['Id']
    workspace_name = row['Name']
    
    try:
        df_reports = fabric.list_reports(workspace_id)
        if not df_reports.empty:
            df_reports['ReportWorkspaceName'] = workspace_name
            df_reports['ReportWorkspaceId'] = workspace_id
            all_reports.append(df_reports)
    except Exception as e:
        print(f"Error fetching reports for workspace '{workspace_name}': {e}")

# Combine all reports into one DataFrame
if all_reports:
    df_all_reports = pd.concat(all_reports, ignore_index=True)
    df_all_reports=df_all_reports.merge(df_SemanticModels[["Id", "Display Name", "Workspace Id","Name"]], left_on="Dataset Id", right_on="Id", suffixes=('', '_Dataset'))
    df_all_reports=df_all_reports[['Id','Report Type','Name','ReportWorkspaceId','ReportWorkspaceName','Dataset Id','Display Name','Dataset Workspace Id','Name_Dataset']]
    df_all_reports=df_all_reports.rename(
    columns={
        'Id': 'Report Id',
        'Name': 'Report Name',
        'ReportWorkspaceId' : 'Report Workspace Id',
        'ReportWorkspaceName' : 'Report Workspace Name',
        'Dataset Id' : 'Semantic Model Id',
        'Display Name' : 'Semantic Model Name',
        'Dataset Workspace Id' : 'Semantic Model Workspace Id',
        'Name_Dataset' : 'Semantic Model Workspace Name'
    }
) #Rename the column names

# Filter based on Parameter Value

if Param_SemanticModelId.strip(): 
    df_all_reports = df_all_reports[df_all_reports["Semantic Model Id"] == Param_SemanticModelId]
else:
    df_all_reports = df_all_reports #Get all Report Details

if not df_all_reports.empty:
    print(f"Total reports found: {len(df_all_reports)}")
    display(df_all_reports)
else:
    print("No reports found across workspaces.")

# Create DAG using networkx
G = nx.DiGraph()
node_colors = {}

for _, row in df_all_reports.iterrows():
    source_node = f"{row['Report Name']} ({row['Report Id']}) ({row['Report Workspace Name']})"
    target_node = f"{row['Semantic Model Name']} ({row['Semantic Model Id']}) ({row['Semantic Model Workspace Name']})"
    
    G.add_edge(source_node, target_node)
    
    # Assign colors
    node_colors[source_node] = 'lightblue'   # sources in blue
    node_colors[target_node] = 'lightgreen'  # connections in green


#Plot the DAG

plt.figure(figsize=(14, 10))
pos = nx.spring_layout(G, k=0.6, seed=42)

# Draw nodes with custom colors

nx.draw(
    G, pos,
    with_labels=True,
    node_color=[node_colors[node] for node in G.nodes()],
    node_size=3000,
    font_size=8,
    font_weight='bold',
    arrows=True
)

plt.title("Semantic Model Dependency DAG", fontsize=16)
plt.axis('off')
plt.show()